# Dac manipulation

## Check current current values on all dacs if needed

In [2]:
from tergite_acl.utils.hardware_utils import SpiDAC
from tergite_acl.config.coupler_config import coupler_spi_map
try:
    spi = SpiDAC()
except:
    pass
couplers = list(coupler_spi_map.keys())
dacs = [spi.create_spi_dac(coupler) for coupler in couplers]
coupler_dac = dict(zip(couplers,dacs))
print(couplers)
print([dac.current() for dac in dacs])

Connected to: Qblox SPI Rack (serial:None, firmware:{'device': 'v1.6 - May 10 2019 - mt', 'driver': {'version': '0.11.1', 'build': '18/09/2023-09:00:54', 'hash': '977377ad', 'dirty': False}}) in 0.00s
['q16_q17', 'q17_q18', 'q18_q19', 'q19_q20', 'q18_q23', 'q17_q22', 'q16_q21', 'q19_q24', 'q20_q25', 'q21_q22', 'q22_q23', 'q23_q24', 'q24_q25', 'q13_q18', 'q15_q20', 'q14_q19']
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


## Set all dacs to zero if needed

In [3]:
parking_current = 0e-6
for coupler in couplers:
# coupler = couplers[0]
    dac = coupler_dac[coupler]
    spi.set_dac_current(dac, parking_current)
    print(f'Coupler {coupler} is parked at {dac.current()}.')

Finished ramping
 target_current = 0.0
 dac.current() = 0.0
Coupler q16_q17 is parked at 0.0.
Finished ramping
 target_current = 0.0
 dac.current() = 0.0
Coupler q17_q18 is parked at 0.0.
Finished ramping
 target_current = 0.0
 dac.current() = 0.0
Coupler q18_q19 is parked at 0.0.
Finished ramping
 target_current = 0.0
 dac.current() = 0.0
Coupler q19_q20 is parked at 0.0.
Finished ramping
 target_current = 0.0
 dac.current() = 0.0
Coupler q18_q23 is parked at 0.0.
Finished ramping
 target_current = 0.0
 dac.current() = 0.0
Coupler q17_q22 is parked at 0.0.
Finished ramping
 target_current = 0.0
 dac.current() = 0.0
Coupler q16_q21 is parked at 0.0.
Finished ramping
 target_current = 0.0
 dac.current() = 0.0
Coupler q19_q24 is parked at 0.0.
Finished ramping
 target_current = 0.0
 dac.current() = 0.0
Coupler q20_q25 is parked at 0.0.
Finished ramping
 target_current = 0.0
 dac.current() = 0.0
Coupler q21_q22 is parked at 0.0.
Finished ramping
 target_current = 0.0
 dac.current() = 0.0


/home/loke-c/miniforge3/envs/tac/lib/python3.9/site-packages/qblox_instruments/native/spi_rack_modules/s4g_module.py:407: RuntimeWarning: invalid value encountered in scalar divide
  dt = np.abs(val_begin - val) / self._ramp_rate / num_steps


## Set the current of the dac corresponding to a certain coupler to a certain value if needed

In [ ]:
coupler = 'q23_q24'
parking_current = -750e-6
spi.set_dac_current(coupler_dac[coupler], parking_current)
print(f'Coupler {coupler} is parked at {coupler_dac[coupler].current()}.')

## Get the parking current of a certain coupler in Redis if needed

In [4]:
import redis
redis_connection = redis.Redis(decode_responses=True)
couplers = ['q23_q24']
for coupler in couplers: 
    print(f"Coupler parameters {coupler}:")
    redis_config = redis_connection.hgetall(f"transmons:{coupler}")
    for key, value in redis_config.items():
        print(f"{key}: {value}")

Coupler parameters q23_q24:


## Set the parking current of a certain coupler in Redis if needed

In [ ]:
coupler = 'q23_q24'
transmon_parameter = 'parking_current'
for coupler in couplers: 
    redis_connection.hset(f"transmons:{coupler}", f"{transmon_parameter}",parking_current)
parking_current = float(redis_connection.hget(f'transmons:{coupler}', 'parking_current'))
print(parking_current)

# Modulation amplitude calibration

## Configure current sweep range and calculate time if needed
User needs to esimate time per sweep on their own

In [ ]:
from tergite_acl.scripts.monitor import Monitor
import numpy as np
import os
monitor = Monitor()
monitor.node_status('cz_chevron')
# monitor.calibrate_node('cz_chevron')
current_sweep = np.arange(-1200,-600,-10)*1e-6
print(f'{current_sweep = }')
time_per_sweep = 0.1
time = len(current_sweep)*time_per_sweep
print(f'{time = } hours')

Establish connection to SPI and print all current dac values

In [ ]:
import numpy as np
from tergite_acl.utils.hardware_utils import SpiDAC
from tergite_acl.config.coupler_config import coupler_spi_map
try:
    spi = SpiDAC()
except:
    pass
all_couplers = list(coupler_spi_map.keys())
dacs = [spi.create_spi_dac(coupler) for coupler in all_couplers]
coupler_dac = dict(zip(all_couplers,dacs))
print(f'{all_couplers = }')
print([dac.current() for dac in dacs])

from tergite_acl.utils.user_input import qubits,couplers
print(f'curent = {coupler_dac[couplers[0]].current()}')

Sweep bias current and plot (and fit) modeulation amplitude vs modulation frequency of the coupler

In [ ]:
all_results = []
amp = 0.5 #optional if you want to additionally run a chevron (gate time vs frequency) node in the following for loop
half_cz_time = 120e-9
for parking_current in current_sweep:
    print('--------------------------------------------')
    print(f'{parking_current = }')
    print('--------------------------------------------')
    spi.set_dac_current(coupler_dac[couplers[0]], parking_current)
    monitor.calibrate_node('qubit_01_spectroscopy') # calibrate qubit frequency in case it shifted due to bias current
    #monitor.calibrate_node('rabi_oscillations')

    print('++++++++++++++++++++++++++++++++++++++++++++')
    #optionally run chevron node at this bias 
    #monitor.calibrate_node('cz_chevron',cz_pulse_amplitude = amp)
    #all_results.append({'name':monitor.get_name(),'parking_current':parking_current,'amplitude': amp,'result':monitor.get_results()})
    monitor.calibrate_node('cz_chevron_amplitude',cz_pulse_duration = half_cz_time)
    all_results.append({'name':monitor.get_name(),'parking_current':parking_current,'duration':half_cz_time,'result':monitor.get_results()})
    date = all_results[0]['name'][:8]
    path = 'tergite_acl/notebooks/cz/'+date+'/'
    try:
        os.mkdir(path)
    except:
        pass
    with open(path+couplers[0]+'_'+all_results[0]['name']+'_all_results.py', 'w') as f:
        f.write(f"all_results = {all_results}\n")

If you found a good bias point, you run a chevron (if you haven't already) to extract the initial guess gate time and frequency

This should be run at amp you fit from the optimum current in the previous loop

In [ ]:
all_results = []
amp = 0.5 #the amplitude you fit from the best bias point
half_cz_time = 120e-9 #optional if you want to run the amplitude vs frequency at this bias point as well
optimal_bias = -720e-6
print('--------------------------------------------')
print(f'{optimal_bias = }')
print('--------------------------------------------')
spi.set_dac_current(coupler_dac[couplers[0]], optimal_bias)
monitor.calibrate_node('qubit_01_spectroscopy') # calibrate qubit frequency in case it shifted due to bias current
#monitor.calibrate_node('rabi_oscillations')

print('++++++++++++++++++++++++++++++++++++++++++++')
monitor.calibrate_node('cz_chevron',cz_pulse_amplitude = amp)
all_results.append({'name':monitor.get_name(),'parking_current':parking_current,'amplitude': amp,'result':monitor.get_results()})

#optionally run amplitude vs frequency node node at this bias 

# monitor.calibrate_node('cz_chevron_amplitude',cz_pulse_duration = half_cz_time)
# all_results.append({'name':monitor.get_name(),'parking_current':parking_current,'duration':half_cz_time,'result':monitor.get_results()})
date = all_results[0]['name'][:8]
path = 'tergite_acl/notebooks/cz/'+date+'/'
try:
    os.mkdir(path)
except:
    pass
with open(path+couplers[0]+'_'+all_results[0]['name']+'_all_results.py', 'w') as f:
    f.write(f"all_results = {all_results}\n")

## CZ optimization

Manually input the CZ gate time and frequency you extract from the previous chevron and set that to redis

In [ ]:
cz = {'q14_q15': {'parking_current': -0.000720,
'cz_pulse_frequency': 558000000.0,
'cz_pulse_duration':260e-9,
'cz_pulse_amplitude': 0.5,
'cz_dynamic_target': 0,
'cz_dynamic_control': 0,
'tqg_fidelity': 0}}

from tergite_acl.utils.user_input import qubits,couplers
import redis
redis_connection = redis.Redis(decode_responses=True)
print(cz)
for coupler in couplers:
    for key, value in cz[coupler].items():
        redis_connection.hset(f"couplers:{coupler}", key,value)

Start the optimizer

In [ ]:
from tergite_acl.scripts.monitor import Monitor, OptimizeNode
import numpy as np
import os
monitor = Monitor()
monitor.node_status('cz_calibration_ssro')
# monitor.calibrate_node('cz_calibration')
#optimize Node can be ran so far on either "cz_calibration_ssro_node" or "tqg_randomized_benchmarking_interleaved_ssro" node
#if you choose to run with "tqg_randomized_benchmarking_interleaved_ssro"
optimize = OptimizeNode('tqg_randomized_benchmarking_interleaved_ssro',trails = 80)

#if you choose to run with "cz_calibration"
optimize = OptimizeNode('cz_calibration_ssro_node',trails = 80, optimize_swap=False, number_of_cz = 3)

optimize_results = optimize.optimize_node()

run validate_cz() to run once with the optimized parameters. The optimize_node() automatically runs this but you can run it again anyways

In [ ]:
results = optimize.validate_cz()

These are the best parameters. Note that this is just the epsilon over the input CZ values

In [ ]:
optimize_results.best_trial,optimize_results.best_params

Visualize the convergence to solution

In [ ]:
import optuna
optuna.visualization.plot_optimization_history(optimize_results)